In [1]:
from redcap import Project
import pandas as pd
import datetime

#REDCap API configuration
api_url = 'https://ruff.mrc.gm:8443/redcap/api/'
api_key = 'D6807C091B1FC14F57CC9B2972D0B398'
project = Project(api_url, api_key)

#REDCap API configuration for Sensitization
sen_api_url = 'https://redcap.mrc.gm:8443/redcap/api/'
sen_api_key = 'B5C8DD01C47A66E81E154A8AC735D281'
sen_project = Project(sen_api_url,sen_api_key)


In [2]:
################################################# SENSITIZED WOMEN ##############################################################
sensitization_data = sen_project.export_records(forms=['indigo_sensitisation'])
sensitization_df=pd.DataFrame(sensitization_data)

# Specify the fields you want to extract base on logics.
sen_data=sensitization_df[['in_vdate','redcap_event_name','redcap_repeat_instrument','in_kiang_number','in_pregnant','in_study_explained','in_icd_given','in_outcome','in_consent']]
sen_df=pd.DataFrame(sen_data)

#visit date as a date data type
sen_df['in_vdate'] = pd.to_datetime(sen_data['in_vdate'], errors='coerce')

#participants sensitised.
sen_df = sen_df[
    (sen_df['redcap_event_name']=="indigo_sensitisati_arm_2") &
    (sen_df['in_study_explained']=="1")
]

sen_df.to_csv('Terminations_req.csv', index=False)


#participants sensitised and interested.
int_sen = sen_df[
    (sen_df['in_outcome']=="1")
]
int_sen=pd.DataFrame(int_sen)

#participants sensitised and Not interested.
Nint_sen = sen_df[
    (sen_df['in_outcome']=="2")
]
Nint_sen=pd.DataFrame(Nint_sen)

#participants sensitised and undecided.
Und_sen = sen_df[
    (sen_df['in_outcome']=="3")
]
Und_sen=pd.DataFrame(Und_sen)

#participants sensitised and Others.
others_sen = sen_df[
    (sen_df['in_outcome']=="4")
]
others_sen=pd.DataFrame(others_sen)

#Count od different sensitization categories
sensitized_count = len(sen_df)
interested = len(int_sen)
uninterested = len(Nint_sen)
undecided = len(Und_sen)
Others = len(others_sen)
print("Total number of sensitized participants:", sensitized_count)
print("Total number of sensitized(intrested):", interested)
print("Total number of sensitized(Uninterested):", uninterested)
print("Total number of sensitized(Undecided):", undecided)
print("Total number of sensitized(Others):", Others)




Total number of sensitized participants: 725
Total number of sensitized(intrested): 680
Total number of sensitized(Uninterested): 2
Total number of sensitized(Undecided): 1
Total number of sensitized(Others): 39


In [3]:
################################################# CONSENT##############################################################
consent_data = project.export_records(forms=['consent'])
consent_df=pd.DataFrame(consent_data)

# Specify the fields you want to extract base on logics.
con_data=consent_df[['con_participantid_q1','redcap_event_name','redcap_repeat_instrument','con_date_q5','con_dob_q4','con_consent_obtained_q6','con_why_not_consent_q6a','con_language_q7','con_eden_participant_q10','con_cluster_q12',]]
con_df=pd.DataFrame(con_data)

#visit date as a date data type
con_df['con_date_q5'] = pd.to_datetime(con_df['con_date_q5'], errors='coerce')
con_df['con_dob_q4'] = pd.to_datetime(con_df['con_dob_q4'], errors='coerce')

#participants consented.
con_df = con_df[
    (con_df['redcap_event_name']=="recruitment_arm_1") &
    (con_df['con_date_q5'].notnull())
]


#participants consented and interested.
con_not_int = con_df[
    (con_df['con_consent_obtained_q6']=="0")
]
con_not_int=pd.DataFrame(con_not_int)

consented_count = len(con_df)
con_not_interested = len(con_not_int)
print("Total number of consented participants:", consented_count)
print("Total number of consented participants and not interested:", con_not_interested)


Total number of consented participants: 670
Total number of consented participants and not interested: 0


In [4]:
################################################# STUDY TERMINATION ##############################################################
termination_data = project.export_records(forms=['study_termination'])
termination_data=pd.DataFrame(termination_data)

# Specify the fields you want to extract base on logics
df_withdrw=termination_data[['con_participantid_q1','st_date','st_reason','st_lastactivity','st_specify','redcap_event_name']]
df_withdrw=pd.DataFrame(df_withdrw)

#st_date as a date data type
df_withdrw['st_date'] = pd.to_datetime(df_withdrw['st_date'], errors='coerce')


df_withdrw = df_withdrw[
    (df_withdrw['redcap_event_name'] == 'end_of_study_arm_1') &
    (df_withdrw['st_date'].notnull())
]
# ################################################################
# #LIST OF TERMINATIONS AND REASON FOR TERMINATION#
# #################################################################
# term_mapping = {
# '1':'Screening',
# '2':'Week 20',
# '3':'Week 28',
# '4':'Week 36',
# '5':'Delivery',
# '6':'Baby Check',
# '7':'Infant Screening',
# '8':'Month 1',
# '9': 'Month 2',
# '10': 'Month 3',
# '11': 'Month 4',
# '12': 'Month 5',
# '13': 'Month 6'
# }

# # Assuming sen_log_data is your DataFrame
# df_withdrw['st_lastactivity'] = df_withdrw['st_lastactivity'].map(term_mapping)
# #----------------------------------------------------------------------------------------------------#
# reason_mapping = {
# '1':'Consent Withdrawn',
# '2':'Moved Away',
# '3':'Refusal to take IP',
# '4':'Maternal Screening Failure',
# '5':'Infant Screening Failure',
# '6':'Repeated Clinic Visit default',
# '7':'Taking Other Micronutrient Supplements',
# '8':'SAE',
# '9': 'Lost to follow-up (LFU)',
# '10': 'Clinician/ Investigator''s discretion',
# '11': 'DSM recommendation',
# '12': 'Other'
# }

# # Assuming sen_log_data is your DataFrame
# df_withdrw['st_reason'] = df_withdrw['st_reason'].map(reason_mapping)

# #################################################################
# df_withdrw.to_csv('Terminations_req.csv', index=False)
# ####################################################################
# #END#
# ####################################################################



# Assuming you already have termination_df DataFrame defined

# Function to count terminations at a specific activity point
def last_activity_count(term_df, activity_point):
    term_act = term_df[term_df['st_lastactivity'] == activity_point]
    return len(term_act)

# Assuming you have filtered your term_df DataFrame appropriately before calling the function

# Example usage:
# Count terminations at activity point 1
term_point_1 = last_activity_count(df_withdrw, '1')
print("Total terminated at screening:", term_point_1)

# Count terminations at activity point 2 (screening)
term_screening = last_activity_count(df_withdrw, '2')
print("Total terminated at week 20:", term_screening)

# Count terminations at activity point 3 (some other activity point)
term_point_3 = last_activity_count(df_withdrw, '3')
print("Total terminated at week 28:", term_point_3)

# Count terminations at activity point 4 (some other activity point)
term_point_4 = last_activity_count(df_withdrw, '4')
print("Total terminated at week 36:", term_point_4)

# Count terminations at activity point 5 (some other activity point) 8 13 19
term_point_5 = last_activity_count(df_withdrw, '5')
print("Total terminated Delivery:", term_point_5)

term_point_8 = last_activity_count(df_withdrw, '8')
print("Total terminated month 1:", term_point_8)

term_point_13 = last_activity_count(df_withdrw, '13')
print("Total terminated at month 6:", term_point_13)

term_point_19 = last_activity_count(df_withdrw, '19')
print("Total terminated at month 12:", term_point_19)



Total terminated at screening: 198
Total terminated at week 20: 8
Total terminated at week 28: 10
Total terminated at week 36: 3
Total terminated Delivery: 6
Total terminated month 1: 7
Total terminated at month 6: 1
Total terminated at month 12: 0


In [5]:
################################################# RECRUITMENT/ WEEK 20 ##############################################################
recruitment_data = project.export_records(forms=['recruitment'])
recruitment_df=pd.DataFrame(recruitment_data)

# Specify the fields you want to extract base on logics.
rec_data=recruitment_df[['con_participantid_q1','redcap_event_name','redcap_repeat_instrument','rvdate','rage','ga_at_recruitment','pregnancyless20w','rsingleton_pregnancy','williness_daily_capsule','if_the_primary_language_of','mother_has_intention_to_re','history_chronic_disease','hb_result_from_lab','severe_anaemia','eligible']]
rec_df=pd.DataFrame(rec_data)

#visit date as a date data type
rec_df['rvdate'] = pd.to_datetime(rec_df['rvdate'], errors='coerce')

#participants recruitment.
rec_df = rec_df[
    (rec_df['redcap_event_name']=="week_20_arm_1") &
    (rec_df['rvdate'].notnull())
]

# rec_eligible = rec_df[
#     (rec_df['eligible']=='1') | (rec_df['eligible']=='')
# ]
# rec_eligible.to_csv('eligible1.csv')

#participants recruited and eligible.
rec_not_eligible = rec_df[
    (rec_df['eligible']=='1')
]


rec_not_eligible=pd.DataFrame(rec_not_eligible)

rec_count = len(rec_df)
rec_not_eligible = len(rec_not_eligible)
print("Total number of recruited participants:", rec_count)
print("Total number of recruited participants and not eligible:", rec_not_eligible)

Total number of recruited participants: 447
Total number of recruited participants and not eligible: 415


In [6]:
################################################# RANDOMIZATION ##############################################################
randomization_data = project.export_records(forms=['randomization'])
randomization_df=pd.DataFrame(randomization_data)

# Specify the fields you want to extract base on logics.
rand_data=randomization_df[['con_participantid_q1','redcap_event_name','redcap_repeat_instrument','ran_date','rnd_arm']]
rand_df=pd.DataFrame(rand_data)

#visit date as a date data type
rand_df['ran_date'] = pd.to_datetime(rand_df['ran_date'], errors='coerce')

#participants recruitment.
ran_df = rand_df[
    (rand_df['redcap_event_name']=="week_20_arm_1") &
    (rand_df['rnd_arm'].notnull() | (rand_df['rnd_arm'] !='')) &
    (rand_df['ran_date'].notnull())
    ]

#ran_df.to_csv('Rand1.csv')
rand_count = len(ran_df)
print("Total number of Randomized participants:", rand_count)

Total number of Randomized participants: 415


In [7]:
################################################# WEEK 28 ##############################################################
USS_data = project.export_records(forms=['ultrasound_scan'])
USS_df=pd.DataFrame(USS_data)

# Specify the fields you want to extract base on logics.
ultsc_data=USS_df[['con_participantid_q1','redcap_event_name','redcap_repeat_instrument','ultsc_date','pregnancy_confirmed_by_ult_q2','ultsc_cardiac_act_q3','ga_assessby','ga_weeks','ga_days']]
ultsc_df=pd.DataFrame(ultsc_data)

#visit date as a date data type
ultsc_df['ultsc_date'] = pd.to_datetime(ultsc_df['ultsc_date'], errors='coerce')

#participants recruitment.
wk28_ultsc_df = ultsc_df[
    (ultsc_df['redcap_event_name']=="week_28_arm_1") &
    (ultsc_df['ultsc_date'].notnull())
]

wk28_ultsc_count = len(wk28_ultsc_df)
print("Total week 28 visits:", wk28_ultsc_count)

#wk28_ultsc_df.to_csv('eligible28.csv')

Total week 28 visits: 342


In [8]:
################################################# WEEK 36 ##############################################################

#participants recruitment.
wk36_ultsc_df = ultsc_df[
    (ultsc_df['redcap_event_name']=="week_36_arm_1") &
    (ultsc_df['ultsc_date'].notnull())
]

wk36_ultsc_count = len(wk36_ultsc_df)
print("Total week 36 visits:", wk36_ultsc_count)

Total week 36 visits: 261


In [9]:
################################################# DELIVERY ##############################################################
#extract delivery data
delivery_data = project.export_records(forms=['delivery'])
delivery_df=pd.DataFrame(delivery_data)

# Specify the fields you want to extract base on logics
delv_data=delivery_df[['con_participantid_q1','del_vdate_q1','del_place_q2','del_lbonsetdatetime_q4','del_gestage','del_ddate_q7','redcap_event_name']]
delv_df=pd.DataFrame(delv_data)

#del_date as a date data type
delv_df['del_ddate_q7'] = pd.to_datetime(delv_df['del_ddate_q7'], errors='coerce')


delivered_df = delv_df[
    (delv_df['redcap_event_name'] == 'delivery__postpart_arm_1') &
    (delv_df['del_ddate_q7'].notnull())
]



delv_count = len(delivered_df)
print("Total Deliveries:", delv_count)

Total Deliveries: 232


In [10]:
################################################# MONTHLY VISIT FUNCTION ##############################################################
Postpartum_mothers = project.export_records(forms=['anthropometry_and_vital_signs','maternal_blood_sample_collection','maternal_urine_sample_collection','breast_milk_sample_collection_d7cd08'])
Postpartum_df=pd.DataFrame(Postpartum_mothers)

import pandas as pd

# Assuming the Postpartum_mothers data is already loaded and Postpartum_df is created as described

def filter_postpartum_data(event_name):
    # Filtering the DataFrame based on the provided event name and other conditions
    filtered_df = Postpartum_df[
        (Postpartum_df['redcap_event_name'] == event_name) &
        (Postpartum_df['ma_temp_q5'].notnull()) &
        (Postpartum_df['matbld_3mledtacollected_q3'].notnull()) &
        (Postpartum_df['full_urinesample_collected'].notnull()) &
        (Postpartum_df['bmc_10ml_sample'].notnull())
    ]
    return filtered_df


# Month 1 
m1_Postpartum_df = filter_postpartum_data("month_1_arm_1")
m1_Postpartum_count = len(m1_Postpartum_df)
print("Total month 1 visit", m1_Postpartum_count)

# Month 6 
m6_Postpartum_df = filter_postpartum_data("month_6_arm_1")
m6_Postpartum_count = len(m6_Postpartum_df)
print("Total month 6 visit", m6_Postpartum_count)

# Month 12 
m12_Postpartum_df = filter_postpartum_data("month_12_arm_1")
m12_Postpartum_count = len(m12_Postpartum_df)
print("Total month 12 visit", m12_Postpartum_count)

Total month 1 visit 181
Total month 6 visit 92
Total month 12 visit 22
